In [ ]:
# Python script to concatenate, pre-process & sort smrx files with kilosort 2, 2.5, 3

import spikeinterface as si
import spikeinterface.extractors as se
import spikeinterface.sorters as ss
import spikeinterface.toolkit as st
import spikeinterface.widgets as sw
import spikeinterface.exporters as sx
import probeinterface as pi
from pathlib import Path
from pprint import pformat
import datetime
import os
import matplotlib.pyplot as plt
from pynwb import NWBHDF5IO

# the path where smrx files can be found
base_path = Path(
    r"/mnt/sds-hd/sd19b001/PainData/Corrected_Channel_Map/L6/Cortex/20.8.21/KS2"
)
# the location of the probe file
probe_file = Path(r"/mnt/sds-hd/sd19b001/Liam/probes/cambridge_neurotech_H3.prb")
# the path where the output (recording, kilosort, phy, waveforms) should go
output_path = Path(r"/mnt/sds-hd/sd19b001/Liam/Ross_May_3")

# a print function that timestamps the output
def tprint(string=""):
    print(f"|| {datetime.datetime.now():%H:%M:%S} || {string}")

In [ ]:
# load recording, sorting, waveforms:
saved_recording = si.core.load_extractor(output_path / "SI_recording")
sorting = se.read_kilosort(output_path / "tmp_sorting/recording_0/kilosort2")

In [ ]:
wf_path = str(output_path / "wf_ks2")
wf = si.extract_waveforms(
    saved_recording,
    sorting,
    folder=wf_path,
    load_if_exists=True,
    max_spikes_per_unit=None,
    n_jobs=16,
    total_memory="1G",
    progress_bar=True,
)  # , overwrite=True

In [ ]:
from mease_lab_to_nwb import CEDNWBConverter

In [ ]:
# use small text smrx file for now as very slow to convert to nwb
file_recording = "../tests/data/RhdD_H5_Mech+Laser.smrx"
source_data = dict(
    CEDRecording=dict(file_path=file_recording),
    CEDStimulus=dict(file_path=file_recording),
    Elabftw=dict(experiment_id=388),
    Kilosort=dict(folder_path=str(output_path / "tmp_sorting/recording_0/kilosort2")),
)
converter = CEDNWBConverter(source_data)

In [ ]:
# run converter
file_nwb = "kstest.nwb"
converter.run_conversion(nwbfile_path=file_nwb, save_to_file=True, overwrite=True)

In [ ]:
# append units to nwb file
from nwb_conversion_tools import spikeinterface

# open in append mode
io = NWBHDF5IO(file_nwb, "r+")
nwbfile = io.read()
# add units to nwbfile
spikeinterface.add_units(sorting=sorting, nwbfile=nwbfile)
# write changes to disk
io.write(nwbfile)
io.close()

In [ ]:
# read nwb file
io = NWBHDF5IO(file_nwb, "r")
nwbfile = io.read()

In [ ]:
?spikeinterface.add_units

In [ ]:
spikeinterface.add_electrode_groups(recording=saved_recording, nwbfile=nwbfile)

In [ ]:
spikeinterface.add_electrodes(recording=saved_recording, nwbfile=nwbfile)

In [ ]:
# spikeinterface.add_electrical_series(recording=saved_recording, nwbfile=nwbfile, write_as='processed')

In [ ]:
from nwbwidgets import nwb2widget

nwb2widget(nwbfile)

In [ ]:
io.close()

In [ ]:
from mease_lab_to_nwb.convert_ced.cednwbconverter import quick_write
from datetime import datetime

In [ ]:
quick_write("a", "b", datetime(1971, 1, 1), save_path="try.nwb", sorting=sorting)